In [1]:
import toolbox_path
import os
import tqdm

%matplotlib inline

from planning_toolbox.parser import parse_file
from planning_toolbox.solvers import *
from planning_toolbox.determinization import *

from utils import *

In [2]:
folder = "../benchmarks/imagine_simple/"
domain = parse_file(folder+"domain.pddl", "domain")
problems = sorted(folder+f for f in os.listdir(folder) if PROBLEM_FILE_RE.match(f))
print(domain)

(define (domain imagine)

(:requirements :adl :rewards :probabilistic-effects)

(:types pliers-point lever-point suction-point - affordance
screw - removable-component
component mode affordance-confidence affordance tool side - object
screwdriver - tool
removable-component static-component - component)

(:constants top bottom front back left right - side low medium high - affordance-confidence scara power no-mode - mode flat-sd star-sd - screwdriver hammer suction-tool pliers cutter no-tool - tool)

(:predicates
  (has-affordance ?c - removable-component ?a - affordance)
  (has-confidence ?a - affordance ?c - affordance-confidence)
  (broken-component ?c - removable-component)
  (broken-tool ?t - tool)
  (connected ?c1 ?c2 - component)
  (clear ?c - removable-component)
  (current-mode ?m - mode)
  (current-side ?s - side)
  (current-tool ?t - tool)
  (held)
  (fixed-by ?c - removable-component ?s - screw)
  (loose ?c - removable-component)
  (partially-occludes ?c1 ?c2 - component)
  

In [3]:
determinizer = AlphaCostLikelihoodDeterminizer(alpha=0, round_=4)
determinizer.set_domain(domain)
print(determinizer.determinized_domain)

(define (domain imagine)

(:requirements :adl)

(:types pliers-point lever-point suction-point - affordance
screw - removable-component
component mode affordance-confidence affordance tool side - object
screwdriver - tool
removable-component static-component - component)

(:constants top bottom front back left right - side low medium high - affordance-confidence scara power no-mode - mode flat-sd star-sd - screwdriver hammer suction-tool pliers cutter no-tool - tool)

(:predicates
  (has-affordance ?c - removable-component ?a - affordance)
  (has-confidence ?a - affordance ?c - affordance-confidence)
  (broken-component ?c - removable-component)
  (broken-tool ?t - tool)
  (connected ?c1 ?c2 - component)
  (clear ?c - removable-component)
  (current-mode ?m - mode)
  (current-side ?s - side)
  (current-tool ?t - tool)
  (held)
  (fixed-by ?c - removable-component ?s - screw)
  (loose ?c - removable-component)
  (partially-occludes ?c1 ?c2 - component)
  (removed-non-verified ?c - remov

In [6]:
solver = FDPlanner(search="astar(hmax())")
# solver = FFPlanner(s=3, w=1)
problem = parse_file(problems[0], "problem", domain)

alphas = []
plan_cost = []
plan_prob = []
plans = []

for alpha in tqdm.tqdm(list(linspace(1, 6, 100))):
    determinizer.alpha = alpha
    determinizer.reset()
    deterministic_problem = determinizer(problem)
    result = solver(deterministic_problem, timeout=30)
    if result["plan-found"]:
        cost = result["total-cost"]
        prob, plan = determinizer.process_plan_trace(result["plan"])
        alphas.append(alpha)
        plan_cost.append(cost)
        plan_prob.append(prob)
        plans.append(plan)
    else:
        print(result["stdout"])
        print(result["stderr"])
        break


  3%|▎         | 3/100 [00:03<01:58,  1.22s/it]


KeyboardInterrupt: 

In [ ]:
plot_results_actl(alphas, plan_prob, [len(p) for p in plans], [c/10000 for c in plan_cost])

In [ ]:
print_plan("1st plan", plans[0])
print_plan("2nd plan", plans[int((0.25 - 0)*99/6)])
print_plan("3rd plan", plans[int((1 - 0)*99/6)])
print_plan("4th plan", plans[int((5.8 - 0)*99/6)])